<a href="https://colab.research.google.com/github/shanxi85/chatbot/blob/master/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#libraries needed for NLP
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

#Libararies needed for TensorFlow processing
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

W0625 06:27:13.744262 139713248307072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0625 06:27:13.746653 139713248307072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0625 06:27:13.770741 139713248307072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0625 06:27:13.784030 139713248307072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0625 06:27:13.803727 139713248307072 deprecation_wrapper.py:119] From /usr/local/lib/

In [40]:
from google.colab import files
files.upload()

Saving intent.json to intent (1).json


{'intent.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "mopeds",\r\n         "patterns": ["W

In [0]:
#import chat-bot intents file
with open('intent.json') as json_data:
    intents = json.load(json_data)

In [42]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['Which mopeds do you have?',
    'What kinds of mopeds are there?',
    'What do you rent?'],
   'responses': ['We rent Yamaha, Piaggio and Vespa mopeds',
    'We have Piaggio, Vespa and Yamaha mopeds'

In [0]:
words = []
classes = []
documents = []
ignore = ['?']

#loop through each sentence in the intent's patters
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each and every word in the sentence
        w=nltk.word_tokenize(pattern)
        
        #add word to the word list
        words.extend(w)
        
        #add word(s) to documents
        documents.append((w, intent['tag']))
        
        #add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [46]:
#perform stemming and lower each word as well as remove duplicates

words=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words=sorted(list(set(words)))

#remove duplicate classes
classes=sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [0]:
#create training data
training=[]
output=[]

#create an empty array for output
output_empty=[0]*len(classes)

#create training set bag of words for each sentence
for doc in documents:
  #initialize bag of words
  bag=[]
  #list of tokenized words for the pattern
  pattern_words=doc[0]
  #stemming each word
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
  #create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    
  #output is 1 for current tag and o for the rest of other tags
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1
    
  training.append([bag,output_row])
    
#shuffling features and turning it into np.array
random.shuffle(training)
training=np.array(training)
  
#creating training lists
train_x =list(training[:,0])
train_y =list(training[:,1])

In [52]:
#resetting underlying graph data
tf.reset_default_graph()

#building neural network
net=tflearn.input_data(shape=[None, len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]), activation='softmax')
net=tflearn.regression(net)

#defining model and setting up tensorboard
model=tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 1.19590 | time: 0.023s
| Adam | epoch: 1000 | loss: 1.19590 - acc: 0.6418 -- iter: 24/27
Training Step: 4000  | total loss: 1.11426 | time: 0.028s
| Adam | epoch: 1000 | loss: 1.11426 - acc: 0.7098 -- iter: 27/27
--


In [0]:
import pickle
pickle.dump({'word':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open("training_data","wb"))

In [0]:
#restoring all the data structures
data = pickle.load(open("training_data","rb"))

words   =data['word']
classes =data['classes']
train_x =data['train_x']
train_y =data['train_y']

In [0]:
with open('intent.json') as json_data:
    intents = json.load(json_data)
  

In [57]:
#load the saved model
model.load('./model.tflearn')

W0625 08:32:48.006549 139713248307072 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [0]:
def clean_up_sentence(sentence):
  #tokenize the token
  sentence_words=nltk.word_tokenize(sentence)
  #stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

#return bag of word array:0 and 1 for each word in the bag that exists
def bow(sentence, words, show_details=False):
  #tokenize sentence
  sentence_words=clean_up_sentence(sentence)
  #generating bag of words
  bag=[0] * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w ==s:
        bag[i] = 1
        
        if show_details:
          print("found in bag: %s" % w)
  return (np.array(bag))

In [0]:
ERROR_THRESHOLD = 0.0
def classify(sentence):
  #generate probabilities from the model
  results = model.predict([bow(sentence, words)])[0]
  
  #filter out prediction below a threshold
  results = [[i,r] for i, r in enumerate(results) if r> ERROR_THRESHOLD]
  
  #sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  
  #return tuple of intent and probability
  return return_list

def response(sentence, userID='123', show_details=False):
  results=classify(sentence)
  
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)
  

In [68]:
classify('what are you hours of operation?')

[('mopeds', 0.49776053),
 ('payments', 0.4294796),
 ('hours', 0.056075975),
 ('opentoday', 0.0122866025),
 ('rental', 0.0037733447),
 ('greeting', 0.00051745604),
 ('goodbye', 7.871018e-05),
 ('thanks', 1.5048857e-05),
 ('today', 1.2706401e-05)]

In [69]:
response('Do you take credit cards?')

We accept most major credit cards


In [70]:
response('Bye')

Hi there, how can I help?


In [73]:
response('good day')

Hi there, how can I help?


In [76]:
response('Thank you')

My pleasure
